In [109]:
from igraph import *
from collections import deque
import random

In [110]:
#returns maximum community label
def get_max_label(g):
    maxVertex = max(g.vs, key = lambda v: max(v["communities"]))
    maxLabel = max(maxVertex["communities"])
    return maxLabel

In [111]:
#removes unused spaces between community labels
def normalize_labels(g):
    maxLabel = get_max_label(g)
    currentLabel = 0
    for i in range(maxLabel):
        vertices = g.vs.select(lambda v: i in v["communities"])
        if len(vertices) > 0 and i != currentLabel:
            #remap vertices
            for v in g.vs:
                for idx in v["communities"]:
                    if (idx == i):
                        idx = currentLabel
            
            currentLabel += 1    

In [119]:
#plots full graph
def plot_full(g, filename="output.svg"):
    normalize_labels(g)
    
    #get max community label
    maxLabel = get_max_label(g)
    
    #define palette
    palette = RainbowPalette(maxLabel + 1)
                   
    #map communities to colors
    for v in g.vs:
        v["color"] = palette.get(v["communities"][0])
        label = ""
        for i in v["communities"]:
            label += str(i) + " "

        v["label"] = label
        v["size"] = len(v["communities"]) * 12 + 5
                   
    #plot
    layout = g.layout("lgl")
    visual_style = {}
    visual_style["layout"] = layout
    verticesCount = len(g.vs)
    visual_style["bbox"] = (16 * (verticesCount + 20), 9 * (verticesCount + 20))
    visual_style["margin"] = 20
    plot(g, filename, **visual_style)

In [120]:
#plots all vertices with distance from start_idx lower than depth
def plot_bfs(g, start_idx = 0, depth = 10, filename="output.svg"):
    all_vertices = []
    vertices = deque()
    vertices.append(g.vs[start_idx])
    
    for v in g.vs:
        v["processed"] = False
        v["distance"] = len(g.vs)
        
    g.vs[start_idx]["distance"] = 0
    
    while (len(vertices) > 0):
        current = vertices.popleft()
        current["processed"] = True
        if current["distance"] == depth:
            break
            
        all_vertices.append(current)
        edges = current.all_edges()
        for e in edges:
            if (e.target_vertex["processed"] == False):
                e.target_vertex["distance"] = min(e.target_vertex["distance"], current["distance"] + 1)
                vertices.append(e.target_vertex)
            
        
    
    subgraph = g.subgraph(all_vertices)
    plot_full(subgraph, filename)

In [121]:
#sample
g = Graph.GRG(100, 0.2)
communitiesCount = 10

for v in g.vs:
    count = random.randint(1, 4)
    v["communities"] = []
    for i in range(count):
        v["communities"].append(random.randint(0, communitiesCount))
    
plot_full(g, "full.svg")
plot_bfs(g, 0, 5, "bfs.svg")